In [1]:
# required packages:
# langchain
# openai
# tiktoken
# chromadb

import os
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    Language,
)

In [2]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://###.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "###"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"

In [3]:
# azure openai

azure_openai_embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', chunk_size=1, disallowed_special=())

azure_chat_openai = AzureChatOpenAI(deployment_name='gpt-4')

In [61]:
# load code

# https://github.com/Azure/azure-rest-api-specs-examples
# only use apimanagement, about 400 java files
repo_dir = 'C:\\github\\azure-rest-api-specs-examples\\specification\\apimanagement\\resource-manager\\Microsoft.ApiManagement\\stable\\2021-08-01\\examples-java'

loaders = []
for dirpath, dirnames, filenames in os.walk(repo_dir):
    for filename in filenames:
        if filename.endswith('.java'):
            loader = TextLoader(os.path.join(dirpath, filename), encoding='utf8')
            loaders.append(loader)
            
# index repository

index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Chroma, 
    embedding=azure_openai_embeddings,
    text_splitter=RecursiveCharacterTextSplitter.from_language(language=Language.JAVA),
    vectorstore_kwargs={
        'persist_directory': './db-api-management'
    }
)

index = index_creator.from_loaders(loaders)

# persist vectordb to disk
index.vectorstore.persist()

In [4]:
# or load existing vectordb from disk

vectordb = Chroma(persist_directory='./db-api-management', embedding_function=azure_openai_embeddings)
index = VectorStoreIndexWrapper(vectorstore=vectordb)

In [63]:
answer = index.query('how to create api management', llm=azure_chat_openai)
print(answer)

To create an API Management service, you can use the following sample code in Java using the Azure SDK:

```java
import com.azure.resourcemanager.apimanagement.models.ApiManagementServiceSkuProperties;
import com.azure.resourcemanager.apimanagement.models.SkuType;
import java.util.HashMap;
import java.util.Map;

public final class Main {
    public static void apiManagementCreateService(
        com.azure.resourcemanager.apimanagement.ApiManagementManager manager) {
        manager
            .apiManagementServices()
            .define("apimService1")
            .withRegion("South Central US")
            .withExistingResourceGroup("rg1")
            .withSku(new ApiManagementServiceSkuProperties().withName(SkuType.DEVELOPER).withCapacity(1))
            .withPublisherEmail("foo@contoso.com")
            .withPublisherName("foo")
            .withTags(mapOf("Name", "Contoso", "Test", "User"))
            .create();
    }

    @SuppressWarnings("unchecked")
    private static <T> Map

In [46]:
answer = index.query('how to configure gateway', llm=azure_chat_openai)
print(answer)

To configure a gateway using the Azure API Management SDK, follow these steps:

1. Create an instance of `ApiManagementManager`.
2. Use the `ApiManagementManager` instance to create a new gateway.
3. Define the gateway's settings, such as location data and description.
4. Create and configure a Gateway Hostname Configuration for the gateway.

Here's a sample code to create and configure a gateway:

```java
import com.azure.resourcemanager.apimanagement.ApiManagementManager;
import com.azure.resourcemanager.apimanagement.models.ResourceLocationDataContract;

public final class Main {
    public static void main(String[] args) {
        // Initialize ApiManagementManager instance
        ApiManagementManager manager = ...; // Initialize the manager instance with your configuration

        // Create a new gateway
        apiManagementCreateGateway(manager);

        // Configure the Gateway Hostname
        apiManagementCreateGatewayHostnameConfiguration(manager);
    }

    public stati

In [6]:
answer = index.query('how to create logger', llm=azure_chat_openai)
print(answer)

# notes: it actually find the 2 samples of logger configure

To create a logger, you can use the following sample code snippets as a reference:

1. For an Application Insights Logger:

```java
import com.azure.resourcemanager.apimanagement.ApiManagementManager;
import com.azure.resourcemanager.apimanagement.models.LoggerType;

public static void apiManagementCreateAILogger(ApiManagementManager manager) {
    manager
        .loggers()
        .define("loggerId")
        .withExistingService("rg1", "apimService1")
        .withLoggerType(LoggerType.APPLICATION_INSIGHTS)
        .withDescription("adding a new logger")
        .withCredentials(mapOf("instrumentationKey", "11................a1"))
        .create();
}
```

2. For an Azure Event Hub Logger:

```java
import com.azure.resourcemanager.apimanagement.ApiManagementManager;
import com.azure.resourcemanager.apimanagement.models.LoggerType;

public static void apiManagementCreateEHLogger(ApiManagementManager manager) {
    manager
        .loggers()
        .define("eh1")
        .withExisting

In [72]:
answer = index.query('what resource do I need to create, before I can run api release', llm=azure_chat_openai)
print(answer)

# notes: it actually replies what are the depencencies, which is not clear from the samples.
# It would be greate if follow up question like "what are the code to create them" can be answered correctly.

Before you can run API Release, you need to create an API Management service instance and an API within that service. In the provided sample code, "apimService1" represents the API Management service instance, and "a1" represents the API within that service. Ensure that these resources are created and properly configured before running API Release.
